In [1]:
import sqlite3
import pandas as pd
import joblib
import json
from pathlib import Path

In [2]:
BASE_DIR = Path("..")

DB_PATH = BASE_DIR / "data" / "bike_buyers.db"
MODEL_PATH = BASE_DIR / "models" / "global_best_model.pkl"
OUTPUT_PATH = BASE_DIR / "streamlit" / "ui_config.json"

In [3]:
conn = sqlite3.connect(DB_PATH)

df = pd.read_sql("""
SELECT
    c.gender,
    c.age,
    c.marital_status,
    c.children,
    c.income,
    e.education_level,
    o.occupation_name,
    r.region_name,
    cm.commute_distance,
    h.home_owner,
    h.cars,
    p.purchased_bike
FROM purchases p
JOIN customers c ON p.customer_id = c.customer_id
JOIN education e ON p.education_id = e.education_id
JOIN occupation o ON p.occupation_id = o.occupation_id
JOIN region r ON p.region_id = r.region_id
JOIN commute cm ON p.commute_id = cm.commute_id
JOIN housing h ON p.housing_id = h.housing_id
""", conn)

conn.close()

df.head(), df.shape

(   gender  age marital_status  children  income  education_level  \
 0  Female   42        Married         1   40000        Bachelors   
 1    Male   43        Married         3   30000  Partial College   
 2    Male   60        Married         5   80000  Partial College   
 3    Male   41         Single         0   70000        Bachelors   
 4    Male   36         Single         0   30000        Bachelors   
 
   occupation_name region_name commute_distance home_owner  cars  \
 0  Skilled Manual      Europe        0-1 Miles        Yes     0   
 1        Clerical      Europe        0-1 Miles        Yes     1   
 2    Professional      Europe        2-5 Miles         No     2   
 3    Professional     Pacific       5-10 Miles        Yes     1   
 4        Clerical      Europe        0-1 Miles         No     0   
 
    purchased_bike  
 0               0  
 1               0  
 2               0  
 3               1  
 4               1  ,
 (1000, 12))

In [4]:
X = df.drop(columns=["purchased_bike"])
y = df["purchased_bike"]

X.head()

,gender,age,marital_status,children,income,education_level,occupation_name,region_name,commute_distance,home_owner,cars
0,Female,42,Married,1,40000,Bachelors,Skilled Manual,Europe,0-1 Miles,Yes,0
1,Male,43,Married,3,30000,Partial College,Clerical,Europe,0-1 Miles,Yes,1
2,Male,60,Married,5,80000,Partial College,Professional,Europe,2-5 Miles,No,2
3,Male,41,Single,0,70000,Bachelors,Professional,Pacific,5-10 Miles,Yes,1
4,Male,36,Single,0,30000,Bachelors,Clerical,Europe,0-1 Miles,No,0


In [5]:
categorical_features = X.select_dtypes(include="object").columns.tolist()
numeric_features = X.select_dtypes(exclude="object").columns.tolist()

categorical_features, numeric_features

(['gender',
  'marital_status',
  'education_level',
  'occupation_name',
  'region_name',
  'commute_distance',
  'home_owner'],
 ['age', 'children', 'income', 'cars'])

In [6]:
categorical_options = {
    col: sorted(X[col].dropna().unique().tolist())
    for col in categorical_features
}

categorical_options

{'gender': ['Female', 'Male'],
 'marital_status': ['Married', 'Single'],
 'education_level': ['Bachelors',
  'Graduate Degree',
  'High School',
  'Partial College',
  'Partial High School'],
 'occupation_name': ['Clerical',
  'Management',
  'Manual',
  'Professional',
  'Skilled Manual'],
 'region_name': ['Europe', 'North America', 'Pacific'],
 'commute_distance': ['0-1 Miles',
  '1-2 Miles',
  '10+ Miles',
  '2-5 Miles',
  '5-10 Miles'],
 'home_owner': ['No', 'Yes']}

In [7]:
numeric_ranges = {
    col: {
        "min": int(X[col].min()),
        "max": int(X[col].max())
    }
    for col in numeric_features
}

numeric_ranges

{'age': {'min': 25, 'max': 89},
 'children': {'min': 0, 'max': 5},
 'income': {'min': 10000, 'max': 170000},
 'cars': {'min': 0, 'max': 4}}

In [8]:
ui_config = {
    "categorical_features": categorical_options,
    "numeric_features": numeric_ranges
}

ui_config

{'categorical_features': {'gender': ['Female', 'Male'],
  'marital_status': ['Married', 'Single'],
  'education_level': ['Bachelors',
   'Graduate Degree',
   'High School',
   'Partial College',
   'Partial High School'],
  'occupation_name': ['Clerical',
   'Management',
   'Manual',
   'Professional',
   'Skilled Manual'],
  'region_name': ['Europe', 'North America', 'Pacific'],
  'commute_distance': ['0-1 Miles',
   '1-2 Miles',
   '10+ Miles',
   '2-5 Miles',
   '5-10 Miles'],
  'home_owner': ['No', 'Yes']},
 'numeric_features': {'age': {'min': 25, 'max': 89},
  'children': {'min': 0, 'max': 5},
  'income': {'min': 10000, 'max': 170000},
  'cars': {'min': 0, 'max': 4}}}

In [9]:
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

with open(OUTPUT_PATH, "w") as f:
    json.dump(ui_config, f, indent=4)

OUTPUT_PATH

WindowsPath('../streamlit/ui_config.json')